In [2]:
import mysql.connector
import requests
import pandas as pd
import time

db_config = {
    "host": "localhost",     
    "user": "root",  
    "password": "######", 
    "database": "stocks_analysis"
}


api_key = "YOUR_ALPHA_VANTAGE_API_KEY"


conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()


def fetch_stock_data(symbol):
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": symbol,
        "apikey": api_key,
        "outputsize": "compact",  
        "datatype": "json"
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "Time Series (Daily)" not in data:
        print(f"No data found for {symbol}. Check API key or symbol.")
        return None

    df = pd.DataFrame.from_dict(data["Time Series (Daily)"], orient="index")
    df.index = pd.to_datetime(df.index)  
    df = df.rename(columns={
        "1. open": "open_price",
        "2. high": "high_price",
        "3. low": "low_price",
        "4. close": "close_price",
        "5. volume": "volume"
    })
    
    df["stock_id"] = symbol 
    df = df.reset_index().rename(columns={"index": "trade_date"}) 
    return df


def insert_into_mysql(df):
    if df is None or df.empty:
        return
    
    sql = """
    INSERT INTO market_data (stock_id, trade_date, open_price, high_price, low_price, close_price, volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE 
    open_price = VALUES(open_price), 
    high_price = VALUES(high_price), 
    low_price = VALUES(low_price), 
    close_price = VALUES(close_price), 
    volume = VALUES(volume)
    """

    data = df[['stock_id', 'trade_date', 'open_price', 'high_price', 'low_price', 'close_price', 'volume']].values.tolist()
    
    cursor.executemany(sql, data)
    conn.commit()
    print(f"Inserted {len(df)} rows into MySQL for {df['stock_id'].iloc[0]}")


stock_tickers = ['AAPL', 'NVDA', 'GOOGL', 'META' , 'AMZN', 'MSFT', 'TSLA', 'INTC', 'JPM']


for ticker in stock_tickers:
    time.sleep(12)  
    stock_data = fetch_stock_data(ticker)
    insert_into_mysql(stock_data)


cursor.close()
conn.close()
print("All stock data inserted successfully!")

Inserted 100 rows into MySQL for AAPL
Inserted 100 rows into MySQL for NVDA
Inserted 100 rows into MySQL for GOOGL
Inserted 100 rows into MySQL for META
Inserted 100 rows into MySQL for AMZN
Inserted 100 rows into MySQL for MSFT
Inserted 100 rows into MySQL for TSLA
Inserted 100 rows into MySQL for INTC
Inserted 100 rows into MySQL for JPM
All stock data inserted successfully!
